In [27]:
from neo4j import GraphDatabase
import pandas as pd
import os
from dotenv import load_dotenv
import json

In [7]:
load_dotenv()

True

In [8]:
uri = os.getenv('NEO4J_URL')
username=os.getenv('NEO4J_USER')
password=os.getenv('NEO4J_PASSWORD')

In [9]:
driver = GraphDatabase.driver(uri, auth=(username, password))

In [17]:
# Read your CSV file into a pandas DataFrame
df = pd.read_csv('/Users/karan_mudaliar/Desktop/CS 7290/Project/Data/stock_info_description_embedded.csv')
df.head()

,Ticker,Name,Exchange,About,Encoded_About
0,A,Agilent Technologies,NYSE,Agilent Technologies is a global leader in lif...,"[-0.3952235,-0.09160021,-0.18406898,...,0.2637..."
1,AA,Alcoa Inc.,NYSE,Alcoa Inc. is a global leader in lightweight m...,"[-0.07310738,0.5146998,-0.09491941,...,-0.4811..."
2,AAN,Aaron's Inc,NYSE,Aaron's Inc. is a leading omnichannel provider...,"[-0.45554543,-0.06957607,-0.8179232,...,-0.667..."
3,AAT,American Assets Trust,NYSE,American Assets Trust is a real estate investm...,"[-0.39959103,1.6209936,0.02285,...,-0.60889083..."
4,AAV,Advantage Oil & Gas Ltd,NYSE,Advantage Oil & Gas Ltd. is a Canadian company...,"[-0.26057047,0.97616833,-0.7055656,...,-0.9413..."


In [30]:
def update_node_properties(tx, ticker, encoded_about):
    query = """
    MATCH (c:Company {Ticker: $ticker})
    SET c.embedding = apoc.convert.fromJsonList($encoded_about)
    """
    tx.run(query, ticker=ticker, encoded_about=encoded_about)

In [31]:
with driver.session() as session:
    for index, row in df.iterrows():
        # Ensure 'Encoded_About' is a string representation of a list
        encoded_about = row['Encoded_About']
        # Call the function to update Neo4j node properties
        session.execute_write(update_node_properties, row['Ticker'], encoded_about)


ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke function `apoc.convert.fromJsonList`: Caused by: com.fasterxml.jackson.core.JsonParseException: Unexpected character ('.' (code 46)): expected a valid value (JSON String, Number (or 'NaN'/'+INF'/'-INF'), Array, Object or token 'null', 'true' or 'false')
 at [Source: REDACTED (`StreamReadFeature.INCLUDE_SOURCE_IN_LOCATION` disabled); line: 1, column: 38]}

In [24]:
def fetch_updated_nodes(session):
    query = """
    MATCH (c:Ticker)
    WHERE c.embedding IS NOT NULL
    RETURN c.Ticker AS Ticker, c.embedding AS Embedding
    LIMIT 10
    """
    result = session.run(query)
    for record in result:
        print(f"Ticker: {record['Ticker']}, Embedding: {record['Embedding']}")

with driver.session() as session:
    session.read_transaction(fetch_updated_nodes)

/var/folders/dn/jhd9djgd393gp0dk2lz40m2c0000gp/T/ipykernel_6912/3807910468.py:13: DeprecationWarning: read_transaction has been renamed to execute_read
  session.read_transaction(fetch_updated_nodes)
